In [1]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/ML-Project/PII-DATA'

Mounted at /content/drive


In [2]:
import pandas as pd

# Load the CSV data into a DataFrame
df = pd.read_csv('/content/drive/My Drive/ML-Project/PII-DATA/token-labels.csv')

# Display the first few rows of the DataFrame to verify it's loaded correctly
df


,document,words,labels
0,7,"['Design', 'Thinking', 'for', 'innovation', 'r...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,10,"['Diego', 'Estrada', '\n\n', 'Design', 'Thinki...","['B-NAME_STUDENT', 'I-NAME_STUDENT', 'O', 'O',..."
2,16,"['Reporting', 'process', '\n\n', 'by', 'Gilber...","['O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME..."
3,20,"['Design', 'Thinking', 'for', 'Innovation', '\...","['O', 'O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I..."
4,56,"['Assignment', ':', '\xa0 ', 'Visualization', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...
11236,29429,"['Hello,', ""I'm"", 'Nicholas', 'Moore,', 'a', '...","['O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUDENT',..."
11237,29430,"['Hello,', 'my', 'name', 'is', 'Alexey', 'Novi...","['O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME..."
11238,29431,"['My', 'name', 'is', 'Ludmila', 'Inoue,', 'and...","['O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUD..."
11239,29432,"['Dr.', 'Tu', 'Garcia,', 'a', 'renowned', 'der...","['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'O',..."


In [3]:
id2label = {
    0: 'O',
    1: 'B-NAME_STUDENT',
    2: 'I-NAME_STUDENT',
    3: 'B-URL_PERSONAL',
    4: 'B-EMAIL',
    5: 'B-ID_NUM',
    6: 'I-URL_PERSONAL',
    7: 'B-USERNAME',
    8: 'B-PHONE_NUM',
    9: 'I-PHONE_NUM',
    10: 'B-STREET_ADDRESS',
    11: 'I-STREET_ADDRESS',
    12: 'I-ID_NUM'
}

In [4]:
# import ast
# df['words'] = df['words'].apply(ast.literal_eval)
# df['labels'] = df['labels'].apply(ast.literal_eval)


In [5]:
import re
import ast

def clean_words_and_labels(words, labels):
    # Clean words and remove corresponding labels if the word is just whitespace or empty after cleaning.
    cleaned_words = []
    cleaned_labels = []
    for word, label in zip(words, labels):
        # Apply any specific cleaning necessary for each word
        cleaned_word = re.sub(r'\s+', '', word).strip()
        # Only add words and labels that are not empty after cleaning
        if cleaned_word:
            cleaned_words.append(cleaned_word)
            cleaned_labels.append(label)
    return cleaned_words, cleaned_labels

# Assuming 'words' and 'labels' columns contain string representations of lists
# Convert string representation of a list to actual list
df['words'] = df['words'].apply(ast.literal_eval)
df['labels'] = df['labels'].apply(ast.literal_eval)

# Apply the cleaning function to each pair of words and labels
cleaned_data = df.apply(lambda row: clean_words_and_labels(row['words'], row['labels']), axis=1)

# Separate the cleaned words and labels into their own columns
df['cleaned_words'] = cleaned_data.apply(lambda x: x[0])
df['cleaned_labels'] = cleaned_data.apply(lambda x: x[1])


In [6]:
df.drop(columns=['words', 'labels'], inplace=True)

In [7]:
df

,document,cleaned_words,cleaned_labels
0,7,"[Design, Thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[Diego, Estrada, Design, Thinking, Assignment,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
3,20,"[Design, Thinking, for, Innovation, Sindy, Sam...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4,56,"[Assignment, :, Visualization, Reflection, Sub...","[O, O, O, O, O, O, O, B-NAME_STUDENT, I-NAME_S..."
...,...,...,...
11236,29429,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
11237,29430,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
11238,29431,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
11239,29432,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [8]:
def verify_and_display_alignment(df):
    # Check if each list of 'cleaned_words' has a corresponding list of 'cleaned_labels'
    for index, row in df.iterrows():
        words = row['cleaned_words']
        labels = row['cleaned_labels']
        if len(words) != len(labels):
            print(f"Mismatch found in document {index}: {len(words)} words, {len(labels)} labels")
            break
    else: # This else belongs to the for loop and executes if the loop doesn't break
        print("Each word has a corresponding label.")

    # Display words and labels for the first two 'sentences'
    for index, (words, labels) in enumerate(zip(df['cleaned_words'][2:5], df['cleaned_labels'][2:5])):
        print(f"\nSentence {index + 1}:")
        for word, label in zip(words, labels):
            print(f"{word}: {label}")

# Assuming 'df' is your dataframe and it contains 'cleaned_words' and 'cleaned_labels' columns
verify_and_display_alignment(df)


Each word has a corresponding label.

Sentence 1:
Reporting: O
process: O
by: O
Gilberto: B-NAME_STUDENT
Gamboa: I-NAME_STUDENT
Challenge: O
I: O
received: O
a: O
promotion: O
of: O
being: O
the: O
Regional: O
Controller: O
,: O
along: O
with: O
my: O
actual: O
position: O
of: O
Country: O
CFO: O
.: O
The: O
main: O
responsibility: O
of: O
this: O
new: O
position: O
was: O
to: O
weekly: O
report: O
the: O
results: O
for: O
the: O
week: O
and: O
estimate: O
the: O
final: O
results: O
of: O
the: O
month: O
of: O
4: O
countries: O
and: O
consolidated: O
those: O
.: O
When: O
I: O
was: O
receiving: O
the: O
position: O
,: O
I: O
went: O
to: O
visit: O
my: O
colleague: O
,: O
former: O
Regional: O
Controller: O
,: O
who: O
was: O
promoted: O
to: O
Country: O
CEO: O
and: O
now: O
had: O
interest: O
conflicts: O
of: O
being: O
the: O
controller: O
.: O
The: O
process: O
to: O
consolidate: O
the: O
information: O
of: O
the: O
4: O
countries: O
was: O
that: O
the: O
country: O
controllers: O
se

In [9]:
non_o_word_label_pairs = []

In [10]:
def print_non_O_words_and_labels(df):
    # Get the last five entries in the DataFrame
    last_five = df.iloc[100:105]

    # Iterate over each of the last five entries
    for index, (words, labels) in enumerate(zip(last_five['cleaned_words'], last_five['cleaned_labels'])):
        print(f"Document {index + len(df) - 5}:")
        # Create a list of tuples (word, label) where the label is not 'O'
        non_O_word_label_pairs = [(word, label) for word, label in zip(words, labels) if label != 'O']

        # Print each word with its non-'O' label
        for word, label in non_O_word_label_pairs:
            print(f"{word}: {label}")

# Call the function to print the words and their labels
# This should be run in your local environment
print_non_O_words_and_labels(df)


Document 11236:
Amal: B-NAME_STUDENT
Bhai: I-NAME_STUDENT
Document 11237:
Ahmed: B-NAME_STUDENT
Figo: I-NAME_STUDENT
Document 11238:
Guddu: B-NAME_STUDENT
Singh: I-NAME_STUDENT
Document 11239:
Ahmed: B-NAME_STUDENT
Salem: I-NAME_STUDENT
tps://www.facebook.com/bclark: B-URL_PERSONAL
https://www.youtube.com/channel/UC1ElAcppeuhfet: B-URL_PERSONAL
nYZqnhEXw: I-URL_PERSONAL
Ahmed: B-NAME_STUDENT
Salem: I-NAME_STUDENT
Document 11240:
Fabio: B-NAME_STUDENT
Sameh: I-NAME_STUDENT


In [11]:
id2label = {
    0: 'O',
    1: 'B-NAME_STUDENT',
    2: 'I-NAME_STUDENT',
    3: 'B-URL_PERSONAL',
    4: 'B-EMAIL',
    5: 'B-ID_NUM',
    6: 'I-URL_PERSONAL',
    7: 'B-USERNAME',
    8: 'B-PHONE_NUM',
    9: 'I-PHONE_NUM',
    10: 'B-STREET_ADDRESS',
    11: 'I-STREET_ADDRESS',
    12: 'I-ID_NUM'
}

# Invert the id2label mapping to create a label2id mapping
label2id = {label: id for id, label in id2label.items()}


# Corrected conversion function that can handle a list of lists
def convert_labels_to_integers(label_series, mapping):
    integer_labels = []
    for labels in label_series:
        int_label_row = []
        for label in labels:
            # Check if the label is in the mapping, otherwise ignore it
            if label in mapping:
                int_label_row.append(mapping[label])
        integer_labels.append(int_label_row)
    return integer_labels

# Apply the function to the 'cleaned_labels' column with the new mapping
df['integer_labels'] = convert_labels_to_integers(df['cleaned_labels'], label2id)

df[['cleaned_words', 'integer_labels']]


,cleaned_words,integer_labels
0,"[Design, Thinking, for, innovation, reflexion,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
1,"[Diego, Estrada, Design, Thinking, Assignment,...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[Design, Thinking, for, Innovation, Sindy, Sam...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[Assignment, :, Visualization, Reflection, Sub...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, ..."
...,...,...
11236,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11237,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11238,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11239,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
final = df[['cleaned_words', 'integer_labels']]

In [13]:
final.rename(columns={'integer_labels': 'cleaned_labels'}, inplace=True)

<ipython-input-13-c55765b59f3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.rename(columns={'integer_labels': 'cleaned_labels'}, inplace=True)


In [14]:
final

,cleaned_words,cleaned_labels
0,"[Design, Thinking, for, innovation, reflexion,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
1,"[Diego, Estrada, Design, Thinking, Assignment,...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Reporting, process, by, Gilberto, Gamboa, Cha...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[Design, Thinking, for, Innovation, Sindy, Sam...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[Assignment, :, Visualization, Reflection, Sub...","[0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, ..."
...,...,...
11236,"[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11237,"[Hello,, my, name, is, Alexey, Novikov, and, I...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11238,"[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11239,"[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
from sklearn.model_selection import train_test_split

# Then you can use the train_test_split function
train_val_df, test_df = train_test_split(final, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=(1/9), random_state=42)


In [16]:
train_df

,cleaned_words,cleaned_labels
7245,"[During, my, tenure, as, a, designer, at, Matr...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9681,"[In, the, realm, of, healthcare,, I,, Delun, V...","[0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, ..."
1097,"[STORYTELLING, IS, A, MUST, -, HAVE, TOOL, Cha...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5778,"[Reflection, :, Mind, Mapping, Tool, Describe,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3631,"[Reflection, -, Storytelling, Challenge, I, to...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
10145,"[In, the, realm, of, the, deceased,, where, th...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7254,"[As, a, jeweler,, I've, had, the, privilege, o...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
884,"[VISUALIZATION, Challange, and, selection, The...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5956,"[Reflection, -, Learning, Launching, Challenge...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
val_df

,cleaned_words,cleaned_labels
7714,"[In, the, annals, of, exemplary, nursing, prof...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
9463,"[My, name, is, Kyle, Yamamoto,, and, I, work, ...","[0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6895,"[In, my, role, as, a, pharmacist, with, two, y...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
133,"[Mind, Mapping, Challenge, :, My, challenge, i...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
404,"[Design, Thinking, for, Innovation, by, Univer...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, ..."
...,...,...
4220,"[Visualization, as, a, tool, Challenge, :, Fas...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9578,"[Greetings!, Allow, me, to, introduce, myself,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, ..."
5605,"[Learning, Launch, –, Communication, processes...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4424,"[Challenge, &, Selection, For, my, study, as, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [18]:
test_df

,cleaned_words,cleaned_labels
4170,"[Reflection, –, Mind, Mapping, Challenge, An, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6967,"[Hi,, my, name, is, Carmen, Chen, and, I, work...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
318,"[Federation, By, Sara, Garcia, Challenge, With...","[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3473,"[Example, Reflection, -, Learning, Launch, Cha...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7680,"[Isaac, Scott,, a, dedicated, school, teacher,...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
7234,"[In, a, bustling, city, where, skyscrapers, pi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5417,"[Mind, Mapping, Challenge, &, Selection, As, a...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4110,"[Reflection, Paper, on, Storytelling, Challeng...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7579,"[Hello,, my, name, is, Anil, Morozov., I, was,...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
sampledf = train_df[1:200]

In [20]:
from transformers import BertTokenizerFast
import torch

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def sliding_window_tokenization_and_labels(words, labels, max_len=512, slide_len=256):
    tokenized_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    # Process each sequence with sliding windows
    start_index = 0
    while start_index < len(words):
        end_index = start_index + slide_len
        window_words = words[start_index:end_index]
        window_labels = labels[start_index:end_index]

        # Tokenization and padding to max length
        inputs = tokenizer(
            window_words,
            is_split_into_words=True,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Align labels with the tokenized word IDs, adding -100 for ignored tokens
        word_ids = inputs.word_ids(0)  # Batch index 0 since we're processing one sequence at a time
        window_aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                window_aligned_labels.append(-100)  # For [CLS], [SEP], and subwords
            else:
                label = window_labels[word_idx] if word_idx < len(window_labels) else -100
                window_aligned_labels.append(label)
            previous_word_idx = word_idx

        # Append the tokenized results
        tokenized_inputs['input_ids'].append(inputs['input_ids'].squeeze(0))
        tokenized_inputs['attention_mask'].append(inputs['attention_mask'].squeeze(0))
        tokenized_inputs['labels'].append(torch.tensor(window_aligned_labels, dtype=torch.long))

        # Move start index to the next slide
        start_index += slide_len

    # Stack all the tensors
    tokenized_inputs['input_ids'] = torch.stack(tokenized_inputs['input_ids'])
    tokenized_inputs['attention_mask'] = torch.stack(tokenized_inputs['attention_mask'])
    tokenized_inputs['labels'] = torch.stack(tokenized_inputs['labels'])

    return tokenized_inputs

# Assuming df contains 'cleaned_words' and 'cleaned_labels' with pre-converted integer labels
for index, row in train_df.iterrows():
    tokenized_train_data = sliding_window_tokenization_and_labels(row['cleaned_words'], row['cleaned_labels'])
    # Process the tokenized_data as needed

for index, row in val_df.iterrows():
    tokenized_val_data = sliding_window_tokenization_and_labels(row['cleaned_words'], row['cleaned_labels'])
    # Process the tokenized_data as needed


for index, row in test_df.iterrows():
    tokenized_test_data = sliding_window_tokenization_and_labels(row['cleaned_words'], row['cleaned_labels'])
    # Process the tokenized_data as needed


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
tokenized_train_data

{'input_ids': tensor([[ 101, 1999, 2026,  ...,    0,    0,    0],
         [ 101, 2000, 3445,  ...,    0,    0,    0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([[-100,    0,    0,  ..., -100, -100, -100],
         [-100,    0,    0,  ..., -100, -100, -100]])}

In [22]:
id2label = {
    0: 'O',
    1: 'B-NAME_STUDENT',
    2: 'I-NAME_STUDENT',
    3: 'B-URL_PERSONAL',
    4: 'B-EMAIL',
    5: 'B-ID_NUM',
    6: 'I-URL_PERSONAL',
    7: 'B-USERNAME',
    8: 'B-PHONE_NUM',
    9: 'I-PHONE_NUM',
    10: 'B-STREET_ADDRESS',
    11: 'I-STREET_ADDRESS',
    12: 'I-ID_NUM'
}

In [23]:
def display_tokenized_data(tokenized_data, tokenizer, num_examples=5):
    for i in range(min(num_examples, len(tokenized_data['input_ids']))):
        input_ids = tokenized_data['input_ids'][i]
        labels = tokenized_data['labels'][i]

        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        token_label_pairs = list(zip(tokens, labels))

        print(f"Example {i+1}:")
        for token, label in token_label_pairs:
            # Decode label using inverse_label_map if labels are not -100
            decoded_label = id2label[label.item()] if label != -100 else "Ignored"
            print(f"{token} : {decoded_label}")
        print("\n")


display_tokenized_data(tokenized_train_data, tokenizer, num_examples=3)


Example 1:
[CLS] : Ignored
in : O
my : O
role : O
as : O
an : O
ag : O
##ron : Ignored
##omi : Ignored
##st : Ignored
with : O
over : O
a : O
decade : O
of : O
experience : O
, : Ignored
i : O
have : O
had : O
the : O
privilege : O
of : O
working : O
on : O
numerous : O
projects : O
that : O
have : O
made : O
a : O
positive : O
impact : O
on : O
agricultural : O
practices : O
and : O
crop : O
production : O
. : Ignored
one : O
such : O
project : O
that : O
i : O
am : O
particularly : O
proud : O
of : O
is : O
the : O
implementation : O
of : O
a : O
sustainable : O
irrigation : O
system : O
in : O
a : O
drought : O
- : Ignored
prone : Ignored
region : O
. : Ignored
the : O
project : O
was : O
initiated : O
in : O
a : O
rural : O
area : O
where : O
farmers : O
were : O
struggling : O
to : O
maintain : O
their : O
crops : O
due : O
to : O
erratic : O
rainfall : O
and : O
limited : O
access : O
to : O
water : O
resources : O
. : Ignored
i : O
conducted : O
a : O
thorough : O
assessment : O

In [24]:
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

# Extract the tensors directly from the dictionary
train_input_ids = tokenized_train_data['input_ids']
train_attention_masks = tokenized_train_data['attention_mask']
train_labels = tokenized_train_data['labels']

# Create the TensorDataset
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)

# Then create a DataLoader
batch_size = 32  # Adjust the batch size according to your system's capabilities
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)


In [25]:
val_input_ids = tokenized_val_data['input_ids']
val_attention_masks = tokenized_val_data['attention_mask']
val_labels = tokenized_val_data['labels']

# Create the TensorDataset
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

# Then create a DataLoader
batch_size = 32  # Adjust the batch size according to your system's capabilities

val_dataloader = DataLoader(val_dataset, sampler=RandomSampler(val_dataset), batch_size=batch_size)


In [26]:
test_input_ids = tokenized_test_data['input_ids']
test_attention_masks = tokenized_test_data['attention_mask']
test_labels = tokenized_test_data['labels']

# Create the TensorDataset
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

# Then create a DataLoader
batch_size = 32  # Adjust the batch size according to your system's capabilities

test_dataloader = DataLoader(test_dataset, sampler=RandomSampler(test_dataset), batch_size=batch_size)

